# Libraries 

In [1]:
import pandas as pd

# Data

In [2]:
pitch = pd.read_csv('../datasets/mlb_pitch_stats_19_21_cleaned.csv')
print(pitch.shape)
pitch.head()

(704, 95)


,player_id,last_name,first_name,year,age,ip,bf,ab,h,k%,...,cu_vb_avg,cu_brk_avg,cu_velo_range,2seam%,2seam_velo_avg,2seam_spin_avg,2seam_hb_avg,2seam_vb_avg,2seam_brk_avg,2seam_velo_range
0,425794,Wainwright,Adam,2021,40,196.1,784,725,157,21.6,...,-31.0,9.5,1.6,59.2,87.5,2297,-3.9,-25.0,14.3,2.7
1,425844,Greinke,Zack,2021,38,164.2,670,626,157,17.0,...,NaN,NaN,NaN,45.9,89.0,2225,-2.7,-18.6,16.7,1.3
2,429722,Santana,Ervin,2021,39,61.1,255,232,58,18.8,...,NaN,NaN,NaN,45.3,93.4,2179,-4.5,-17.2,15.0,1.3
3,433589,Petit,Yusmeiro,2021,37,72.1,290,273,63,12.4,...,-21.1,17.3,1.3,68.4,86.5,2134,-3.7,-18.8,18.3,2.1
4,445276,Jansen,Kenley,2021,34,61.2,250,210,33,29.6,...,-14.4,19.2,1.9,84.5,93.0,2578,2.7,-13.6,19.6,2.0


# Data Cleaning

### Nulls and dtypes

In [3]:
pitch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 95 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   player_id         704 non-null    int64  
 1   last_name         704 non-null    object 
 2    first_name       704 non-null    object 
 3   year              704 non-null    int64  
 4   age               704 non-null    int64  
 5   ip                704 non-null    float64
 6   bf                704 non-null    int64  
 7   ab                704 non-null    int64  
 8   h                 704 non-null    int64  
 9   k%                704 non-null    float64
 10  bb%               704 non-null    float64
 11  ba                704 non-null    float64
 12  slg%              704 non-null    float64
 13  obp%              704 non-null    float64
 14  ops               704 non-null    float64
 15  earned_runs       704 non-null    int64  
 16  era               704 non-null    float64
 1